PRUEBA 1

In [14]:
#INGRESAR AUDIO POR MIC
import pyttsx3
from googletrans import Translator
import speech_recognition as sr

def recognize_speech(language="en-US"):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Say something:")
        audio = recognizer.listen(source)

    try:
        text = recognizer.recognize_google(audio, language=language)
        return text
    except sr.UnknownValueError:
        print("Could not understand audio")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

def translate_text(text, target_language="en"):
    translator = Translator()
    translation = translator.translate(text, dest=target_language)
    return translation.text

# Ejemplo de uso
audio_text = recognize_speech()

if audio_text:
    print(f"Texto reconocido: {audio_text}")
    texto_traducido = translate_text(audio_text)
    print(f"Texto traducido: {texto_traducido}")
else:
    print("No se pudo reconocer el audio.")


def text_to_speech(text, rate=120):  # Puedes ajustar el valor de 'rate' según tus preferencias
    engine = pyttsx3.init()
    engine.setProperty('rate', rate)  # Ajusta la velocidad aquí
    engine.say(text)
    engine.runAndWait()

# leer texto traducido
text_to_speech(texto_traducido, rate = 110)


Say something:
Texto reconocido: hola como estas
Texto traducido: Hi, how are you


CODIGO SIN COMENTARIOS

In [21]:
import pyttsx3
from googletrans import Translator, LANGUAGES
import speech_recognition as sr
from langdetect import detect


# Inicializar el reconocedor de voz y el traductor
def recognize_and_translate():
    recognizer = sr.Recognizer()
    translator = Translator()

    with sr.Microphone() as source:
        print("Say something:")
        audio = recognizer.listen(source)

    try:
        text = recognizer.recognize_google(audio, language="auto")
        detected_language = detect(text)
        
        print(f"Detected language: {LANGUAGES[detected_language]}")
        
        print(f"Original text: {text}")

        target_language = "en" if detected_language == "es" else "es"
        translation = translator.translate(text, dest=target_language)
        translated_text = translation.text

        print(f"Translated text: {translated_text}")

        # Leer el texto traducido
        text_to_speech(translated_text, rate=110)

    except sr.UnknownValueError:
        print("Could not understand audio")
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
    except Exception as e:
        print(f"Error: {e}")

# Inicializar el motor de texto a voz
def text_to_speech(text, rate=120):
    engine = pyttsx3.init()
    engine.setProperty('rate', rate)
    # Decir el texto y esperar a que termine
    engine.say(text)
    engine.runAndWait()

# Llamada a la función
recognize_and_translate()


Say something:
Detected language: english
Original text: I'm all ears
Translated text: Soy todo oídos


Exception ignored in: <function BSTR.__del__ at 0x000001A2DF5C4900>
Traceback (most recent call last):
  File "c:\Users\ramir\AppData\Local\Programs\Python\Python312\Lib\site-packages\comtypes\__init__.py", line 699, in __del__
    def __del__(self, _free=windll.oleaut32.SysFreeString):

KeyboardInterrupt: 


CODIGO COMENTADO


In [ ]:
import pyttsx3
from googletrans import Translator, LANGUAGES
import speech_recognition as sr
from langdetect import detect

In [ ]:
# Codigo comentado

def recognize_and_translate():
    # Inicializar el reconocedor de voz y el traductor
    recognizer = sr.Recognizer()
    translator = Translator()

    # Utilizar el micrófono como fuente de audio
    with sr.Microphone() as source:
        print("Say something:")
        audio = recognizer.listen(source)

    try:
        # Reconocer el texto en el audio utilizando Google Speech Recognition
        text = recognizer.recognize_google(audio, language="auto")

        # Detectar automáticamente el idioma del texto usando langdetect
        detected_language = detect(text)
        
        # Imprimir el idioma detectado y el texto original
        print(f"Detected language: {LANGUAGES[detected_language]}")
        print(f"Original text: {text}")

        # Establecer el idioma de destino opuesto al detectado
        target_language = "en" if detected_language == "es" else "es"

        # Traducir el texto al idioma de destino
        translation = translator.translate(text, dest=target_language)
        translated_text = translation.text

        # Imprimir el texto traducido
        print(f"Translated text: {translated_text}")

        # Leer el texto traducido en voz alta
        text_to_speech(translated_text, rate=110)

    except sr.UnknownValueError:
        print("Could not understand audio")
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
    except Exception as e:
        print(f"Error: {e}")

def text_to_speech(text, rate=120):
    # Inicializar el motor de texto a voz
    engine = pyttsx3.init()
    engine.setProperty('rate', rate)
    
    # Decir el texto y esperar a que termine
    engine.say(text)
    engine.runAndWait()

# Llamada a la función principal
recognize_and_translate()


TEST GOOGLE CLOUD ON DEMAND

In [ ]:
from google.cloud import speech_v1p1beta1 as speech
from google.cloud import translate_v2 as translate
import pyttsx3
import keyboard

In [ ]:
# Configura el cliente de Google Cloud Speech
client_speech = speech.SpeechClient()

# Configura el cliente de Google Cloud Translation
client_translate = translate.Client()

def recognize_and_translate():
    # Realiza la transcripción del audio en tiempo real
    streaming_config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",
    )

    streaming_request = speech.StreamingRecognizeRequest(config=streaming_config, single_utterance=True)

    with speech.StreamingMicrophone() as stream:
        requests = (speech.StreamingRecognizeRequest(audio_content=content) for content in stream.generator())
        responses = client_speech.streaming_recognize(streaming_request, requests)

        for response in responses:
            for result in response.results:
                # Obtén el texto reconocido
                text = result.alternatives[0].transcript

                # Detecta automáticamente el idioma del texto
                detected_language = client_translate.detect_language(text)["language"]

                # Establece el idioma de destino opuesto al detectado
                target_language = "en" if detected_language == "es" else "es"

                # Traduce el texto
                translation = client_translate.translate(text, target_language=target_language)
                translated_text = translation["translatedText"]  # Accede a la propiedad "translatedText"

                # Imprime el texto traducido
                print(f"Translated text: {translated_text}")

                # Lee el texto traducido en voz alta
                text_to_speech(translated_text, rate=110)

def text_to_speech(text, rate=120):
    # Inicializa el motor de texto a voz
    engine = pyttsx3.init()
    engine.setProperty('rate', rate)

    # Dice el texto y espera a que termine
    engine.say(text)
    engine.runAndWait()

# Escucha las teclas para activar o desactivar la traducción
keyboard.add_hotkey('ctrl+shift+t', recognize_and_translate)

# Mantén el programa en ejecución
keyboard.wait()
